In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from collections.abc import Callable
import ipympl
import math
import random
import time
import tracemalloc
from functools import wraps
import sys
import gc
%matplotlib ipympl

## Wyznaczanie Błędu

In [ ]:
def maximum_error(A,B):
  n = len(A)
  return max([np.abs(A[i] - B[i]) for i in range(n)])

## Kryteria Stopu

In [ ]:
def progressive_criteria(x, x_next, rho):
  return np.linalg.norm(x_next - x) <= rho

def residual_criteria(A: np.ndarray, b: np.ndarray, x_next: np.ndarray, rho):
  return np.linalg.norm(A @ x_next - b) <= rho

# Implementacja Metody Jakobiego

In [ ]:
def jakobi(A: np.ndarray, b: np.ndarray, rho, x, stop_condition='residual', max_iter=10000):
  D = np.diag(np.diag(A))
  R = A - D
  inv = D / np.power(D[0,0],2) # diagonal matrix inversion, knowing that all the elements on diagonal are equal to k=11
  for i in range(1, max_iter+1):
    x_next = inv @ (b - R @ x)
    if( stop_condition == 'residual' and residual_criteria(A, b, x_next, rho) ):
      return i, x_next
    elif( stop_condition == 'incremental' and progressive_criteria(x, x_next, rho) ):
      return i, x_next

    x = x_next
  
  return i, x_next

## Pomiar Czasu

In [ ]:
def get_time(func, *args, **kwargs):
  gc.collect()
  start_time = time.perf_counter()
  func(*args, **kwargs)
  end_time = time.perf_counter()
  total_time = end_time - start_time
  return total_time

## Wyznaczanie Macierzy

In [ ]:

def get_X_vector(n, precision=np.float64):
  random.seed(42)
  X = np.array([1 if random.randint(1,2) == 1 else -1 for _ in range(n)], dtype=precision)
  return X

k,m = 11,2

def element_function(i, j, n):
  if(i == j):
    return k
  return m/(n-i-j+0.5)

def task_matrix(n, X, precision=np.float64):
  A = np.array([[element_function(i,j,n) for j in range(n)] for i in range(n)], dtype=precision)
  b = A @ X
  return A, b 

def initial_x(A: np.ndarray):
  n = A.shape[0]
  X = np.array([0] * n)

N = 3
PRECISION = np.float64

X = get_X_vector(N, PRECISION)
# A, b = task_2_matrix(N, X, PRECISION)

A, b = (task_matrix(N, X, PRECISION))
print(A)
result = jakobi(A, b, 0.000000001, X)
# np.set_printoptions(precision=1000, suppress=True)
print(X)
print(result)


## Promień spektralny (Spectrial Radius)

In [ ]:
def spectral_radius(A: np.ndarray) -> np.ndarray:
  return np.max(np.abs(np.linalg.eigvals(A)))

def iterative_matrix_spectral(n, X, precision=np.float32):
  A, _ = task_matrix(n,X,precision)
  D = np.zeros((A.shape[0], A.shape[1]))
  np.fill_diagonal(D, A.diagonal())
  R = A - D
  return spectral_radius(-np.linalg.inv(D) @ R)

N = 50
X = get_X_vector(N)
iterative_matrix_spectral(N,X)

print(np.array([iterative_matrix_spectral(i, get_X_vector(i)) for i in (200,500,1000)]))

## Spectral Radius Visualistaion

In [ ]:

# --- Read from CSV and plot ---
def plot_spectral_radius():
  """
  Funkcja wizualizująca promień spektralny na podstawie zapisanych danych.
  """
  data = pd.read_csv('spectral_radius_results.csv')

  sns.set_style("whitegrid")
  plt.figure(figsize=(10, 6))
  plt.plot(data['n'], data['spectral_radius'], 'o', label='Spektralny promień', color='blue')
  plt.xlabel('Rozmiar macierzy (n)')
  plt.ylabel('Spektralny promień')
  plt.yscale('log')
  plt.title('Spektralny promień macierzy iteracji względem rozmiaru n')
  plt.legend()
  plt.grid(True)
  plt.tight_layout()
  plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))

  os.makedirs('plots', exist_ok=True)
  plt.savefig('plots/spectral_radius_plot.png', dpi=300, bbox_inches='tight')
  plt.show()

plot_spectral_radius()


In [ ]:
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

from tqdm.notebook import tqdm

# --- Parameters ---
RHO = [10**k for k in [-2, -3, -4, -5, -7, -10, -15]]
N_SIZES = [3, 5, 10, 15, 20, 50, 100, 200, 500, 750, 1000] 

# RHO = [10**k for k in [-2, -3]]
# N_SIZES = [3, 5]

MAX_ITERATIONS = 10000
OUTPUT_DIR = "data"
X0_TYPES = {
    "zero": lambda n: np.zeros(n),
    "thirty": lambda n: np.full(n, 30.0),
    "hundred": lambda n: np.full(n, 100.0),
}

os.makedirs(OUTPUT_DIR, exist_ok=True)


def run_single_experiment(args):
    stop_condition, x0_type, n, rho = args
    x_true = get_X_vector(n)
    A, b = task_matrix(n, x_true)
    x0 = X0_TYPES[x0_type](n)
    
    start = time.perf_counter()
    iters, x_approx = jakobi(A, b, rho, x0, stop_condition=stop_condition, max_iter=MAX_ITERATIONS)
    total_time = time.perf_counter() - start
    avg_time = total_time / iters if iters > 0 else np.nan
    error = maximum_error(x_true, x_approx)
    
    # print(f"N: {n} x0_type: {x0_type}, stop_condition: {stop_condition}, rho: {rho}")
    
    return {
        "stop_condition": stop_condition,
        "x0_type": x0_type,
        "n": n,
        "rho": rho,
        "iterations": iters,
        "avg_time": avg_time,
        "error": error,
    }


# Generate all tasks
def run_experiments():
    tasks = []
    for stop_condition in ["incremental", "residual"]:
        for x0_type in X0_TYPES:
            for n in N_SIZES:
                for rho in RHO:
                    tasks.append((stop_condition, x0_type, n, rho))

    # Run in parallel with progress bar
    results = Parallel(n_jobs=8)(  # Use fewer cores if Jupyter freezes
        delayed(run_single_experiment)(task) for task in tqdm(tasks)
    )

    # Save results (same as before)
    pd.set_option('display.float_format', '{:.4e}'.format)
    df_all = pd.DataFrame(results)
    # Save results per stop_condition
    
    for stop_condition in ["incremental", "residual"]:
        df_sub = df_all[df_all["stop_condition"] == stop_condition]
        for metric in ["error", "iterations", "avg_time"]:
            df_sub[["rho", "n", "x0_type", metric]].to_csv(
                os.path.join(OUTPUT_DIR, f"heatmap_{stop_condition}_{metric}.csv"),
                index=False,
                float_format='%.5e'
            )

run_experiments()

In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Styl wykresów
sns.set(style="whitegrid")

input_dir = "data"
output_dir = "heatmaps_jakobi"
os.makedirs(output_dir, exist_ok=True)

metrics = ["error", "iterations", "avg_time"]
metrics_names = ["Błąd bezwzględny", "Liczba iteracji", "Średni czas na iterację [s]"]
stop_conditions = ["incremental", "residual"]
stop_conditions_names = ["Kryterium przyrostowe", "Kryterium resydualne"]
x0_types = ["zero", "thirty", "hundred"]
x0_names = {"zero": r"$[0]^{n \times 1}$", "thirty": r"$[30]^{n \times 1}$", "hundred": r"$[100]^{n \times 1}$"}

def plot_heatmap(df, x_col, y_col, x_col_name, y_col_name, value_col, value_col_name, title, filename):
  pivot = df.pivot_table(index=y_col, columns=x_col, values=value_col)
  pivot = pivot[sorted(pivot.columns, reverse=True)]
  pivot = pivot.sort_index()

  plt.figure(figsize=(10, max(4, len(pivot) * 0.5)))

  vmin = pivot.min().min()
  vmax = pivot.max().max()

  # Decide on using log scale if data is all positive and spans multiple orders of magnitude
  use_log = vmin > 0 and vmax / vmin > 1000 or metric == 'iterations' or metric == 'error'

  if use_log:
    from matplotlib.colors import LogNorm
    norm = LogNorm(vmin=max(vmin, 1e-10), vmax=vmax)
    fmt = ".2e" if metric != 'iterations' else ".0f"
  else:
    norm = None
    if vmax - vmin < 1:
      fmt = ".2f"
    elif vmax < 1000:
      fmt = ".1f"
    else:
      fmt = ".0f"

  ax = sns.heatmap(
    pivot,
    cmap="viridis_r",
    annot=True,
    fmt=fmt,
    cbar_kws={"label": value_col_name},
    norm=norm
  )
  
  # Format x-axis labels (rho values) clearly
  ax.set_xticklabels([f"{tick:.0e}" for tick in pivot.columns], rotation=45, ha='right')

  plt.title(title)
  plt.xlabel(x_col_name)
  plt.ylabel(y_col_name)
  plt.tight_layout()
  plt.savefig(filename)
  plt.close()
  print(f"Zapisano: {filename}")

for stop_i, stop in enumerate(stop_conditions):
  for metric_i, metric in enumerate(metrics):
    file_path = os.path.join(input_dir, f"heatmap_{stop}_{metric}.csv")
    if not os.path.exists(file_path):
      print(f"Brakuje pliku: {file_path}")
      continue

    df = pd.read_csv(file_path)

    for x0 in x0_types:
      df_sub = df[df["x0_type"] == x0]

      plot_heatmap(
        df_sub,
        x_col="rho",
        y_col="n",
        x_col_name="Dokładność ρ",
        y_col_name="Rozmiar układu N",
        value_col=metric,
        value_col_name=metrics_names[metric_i],
        title=f"{stop_conditions_names[stop_i]} - {metrics_names[metric_i]} \n $x^{{(0)}}$ = {x0_names[x0]}",
        filename=os.path.join(output_dir, f"{stop}_{metric}_{x0}.png")
      )


## Porównanie

In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Setup
sns.set_style("whitegrid")
input_dir = "data"
output_dir = "comparison_plots"
os.makedirs(output_dir, exist_ok=True)

# Configuration
metrics = ["error", "iterations", "avg_time"]
metrics_names = ["Błąd bezwzględny", "Liczba iteracji", "Średni czas na iterację [s]"]
stop_conditions = ["incremental", "residual"]
stop_conditions_names = {"incremental": "Kryterium przyrostowe", "residual": "Kryterium resydualne"}
x0_types = ["zero", "thirty", "hundred"]
x0_names = {"zero": r"$x^{(0)} = [0]^{n}$", "thirty": r"$x^{(0)} = [30]^{n}$", "hundred": r"$x^{(0)} = [100]^{n}$"}

# Colors for different combinations
colors = {
    ("incremental", "zero"): "#1f77b4",
    ("incremental", "thirty"): "#ff7f0e", 
    ("incremental", "hundred"): "#2ca02c",
    ("residual", "zero"): "#d62728",
    ("residual", "thirty"): "#9467bd",
    ("residual", "hundred"): "#8c564b"
}

def create_metric_comparison_plots():
    """
    Create metric comparison plots with each N value in a separate file.
    Each file shows one metric comparing all stop criteria and starting vectors.
    """
    
    # Load all data
    all_data = []
    for stop in stop_conditions:
        for metric in metrics:
            file_path = os.path.join(input_dir, f"heatmap_{stop}_{metric}.csv")
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                df['stop_condition'] = stop
                df['metric'] = metric
                all_data.append(df)
    
    if not all_data:
        print("Brak danych do wczytania. Upewnij się, że pliki CSV istnieją w katalogu 'data'.")
        return
    
    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Create pivot table for easier plotting
    plot_data = []
    for _, row in combined_df.iterrows():
        plot_data.append({
            'stop_condition': row['stop_condition'],
            'x0_type': row['x0_type'],
            'n': row['n'],
            'rho': row['rho'],
            'metric_type': row['metric'],
            'value': row[row['metric']]
        })
    
    plot_df = pd.DataFrame(plot_data)
    
    # Get all unique N values
    n_values = sorted(plot_df['n'].unique())
    
    # Create plots for each metric and each N value
    for metric_idx, metric in enumerate(metrics):
        metric_data = plot_df[plot_df['metric_type'] == metric]
        
        for n in n_values:
            n_data = metric_data[metric_data['n'] == n]
            
            if n_data.empty:
                continue
            
            # Create figure for this specific metric and N
            plt.figure(figsize=(12, 8))
            
            # Plot each combination of stop criteria and starting vector
            for stop in stop_conditions:
                for x0 in x0_types:
                    subset = n_data[(n_data['stop_condition'] == stop) & (n_data['x0_type'] == x0)].sort_values('rho')
                    
                    if not subset.empty:
                        color = colors[(stop, x0)]
                        label = f"{stop_conditions_names[stop]}, {x0_names[x0]}"
                        
                        plt.plot(subset['rho'], subset['value'], 
                               marker='o', color=color, linewidth=2.5, markersize=8,
                               label=label, alpha=0.8)
            
            plt.xscale('log')
            if metric == 'error' or metric == 'avg_time':
                plt.yscale('log')
            elif metric == 'iterations':
                plt.ylim(top=100, bottom=0)  # Cap iterations plot at 300 (change as needed)

            
            # Set x-axis ticks to show only tested rho values
            tested_rho_values = [10**(-k) for k in [2, 3, 4, 5, 7, 10, 15]]
            actual_rho_values = sorted(n_data['rho'].unique())
            
            plt.xticks(actual_rho_values, [f'$10^{{-{int(-np.log10(rho))}}}$' for rho in actual_rho_values])
            
            plt.xlabel('Dokładność ρ', fontsize=12)
            plt.ylabel(metrics_names[metric_idx], fontsize=12)
            plt.title(f'{metrics_names[metric_idx]} - N = {n}', fontsize=14, fontweight='bold')
            plt.grid(True, alpha=0.3)
            plt.legend(fontsize=10, loc='best')
            
            # Improve layout
            plt.tight_layout()
            
            # Save the plot
            filename = f'metric_comparison_{metric}_N{n}.png'
            plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
            # plt.show()
            print(f"Zapisano: {filename}")
    
    print(f"\nWygenerowano wszystkie wykresy porównawcze!")
    print(f"Liczba wykresów: {len(metrics)} metryk × {len(n_values)} rozmiarów N = {len(metrics) * len(n_values)} plików")

# Run the plotting function
create_metric_comparison_plots()